<a href="https://colab.research.google.com/github/sauravkb94/DNA-Sequencing-Classifier-/blob/main/DNA_Sequencing_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
human_data = pd.read_table('/content/human.txt')

In [ ]:
human_data

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as pyo

In [ ]:
import cv2
from sklearn.model_selection import train_test_split

In [ ]:
geneFamily = cv2.imread("/content/genefamily.PNG")
plt.imshow(geneFamily)

In [ ]:
count = human_data['class'].value_counts()
plt.figure(figsize= (8,4))
sns.barplot(count.index, count.values)
plt.title("Human Class ",fontsize=10)
plt.xlabel('No of Occurances')
plt.ylabel('Category');
plt.show()

In [ ]:
ex.pie(human_data,names='class',title='Propotion Of Human Classes',hole=0.5)

In [ ]:
human_data['length'] = human_data['sequence'].apply(len)

In [ ]:
human_data.hist(column= 'length', by = 'class', bins = 20, figsize=(12,12) );

**Let's define a function to collect all possible overlapping n-grams of a specified length from any sequence string. We will basically apply the n-grams to the complete sequences.**

In [ ]:
allData=[human_data]

In [ ]:
def getKmers(gene, size=4):
    allKmers=""
    for i in range(len(gene) - size + 1):
        allKmers+=gene[i:i+size]
        allKmers+=" "
    return allKmers[:-1]

In [ ]:
# function to convert sequence strings into n-grams words, default size = 6 (hexamer words)
def create_ngrams(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [ ]:
human_data['n-grams'] = human_data['sequence'].apply(create_ngrams)


In [ ]:
human_data.head(10)

**Since we are going to use scikit-learn natural language processing tools to do the k-mer counting, we need to now convert the lists of k-mers for each gene into string sentences of words that the count vectorizer can use. We can also make a y variable to hold the class labels. Let's do that now.**

In [ ]:
human_text = list(human_data['n-grams'])

In [ ]:
for item in range(len(human_text)):
   human_text[item] = ' '.join(human_text[item])

In [ ]:
human_text[1]

In [ ]:
human_data['human_text'] = human_text

In [ ]:
human_data.head(5)

In [ ]:
for i in range(len(allData)):
    allData[i]["human_text"]=allData[i]['sequence'].apply(lambda x: getKmers(x))
    allData[i].drop(columns='sequence',axis=1,inplace=True)

In [ ]:
allData[0]

**Now we will apply the BAG of WORDS using CountVectorizer using NLP**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
for i in range(len(allData)):
    cv = TfidfVectorizer(ngram_range=(5,5))
    X=allData[i]['human_text']
    Y=allData[i]['class']
    X = cv.fit_transform(X)
    allData[i]=[X,Y]

In [ ]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from sklearn import metrics 
from sklearn import metrics

In [ ]:
import tensorflow as tf
ACCURACY_THRESHOLD = 0.95
class endRun(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            self.model.stop_training = True
callbacks = endRun()

In [ ]:
match=[0,0,0]
mismatch=[0,0,0]
for i in range(len(allData)):
    X=allData[i][0]
    y=allData[i][1]
    _,input_sp=X.shape
    x=X.toarray()
    y=pd.get_dummies(y).values
    X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0, train_size = .75)
    model = Sequential()
    model.add(Dense(64,input_shape = (input_sp,), activation = 'relu'))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(7,activation='softmax'))
    print(model.summary())
    
    model.compile(loss='categorical_crossentropy',metrics=['accuracy',])
    his = model.fit(X_train, y_train, epochs=5000, batch_size=128,verbose=1,callbacks=[callbacks])
    yPred=model.predict(X_test)
    for j in range(len(yPred)):
        ans=yPred[j].tolist()
        pred=ans.index(max(ans))
        actual=np.where(y_test[j]==1)[0].tolist()[0]
        if(pred==actual):
            match[i]+=1
            continue
        mismatch[i]+=1

In [ ]:
dataof=["human"]
for i in range(1):
    accuracy=match[i]/(match[i]+mismatch[i])
    print(dataof[i],accuracy)

In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['loss'], c= "red")
plt.plot(h['accuracy'], c = "blue",)

plt.title("loss vs accuracy")
plt.show()
     

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report,accuracy_score

In [ ]:
rf = 'Random Forest Classfier'
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_predicted = rf.predict(X_test)
rf_acc_score = accuracy_score(y_test, rf_predicted) 

In [ ]:
print("\n")
print("Accuracy of Random Forest:",rf_acc_score,'\n')
print(classification_report(y_test,rf_predicted))

In [ ]:
dtc= DecisionTreeClassifier()
dtc.fit(X_train,y_train)
y_pred_dtc = dtc.predict(X_test)

In [ ]:
dtc_acc_score = accuracy_score(y_test, y_pred_dtc)

In [ ]:
print("\n")
print("Accuracy of Decision Tree :",dtc_acc_score,'\n')
print(classification_report(y_test,rf_predicted))

In [ ]:
# Creating the weight of Words model using TfidfTransformer()
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_data = TfidfTransformer().fit_transform(X)

In [ ]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB

In [ ]:
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train, y_train)

In [ ]:
# Creating the Bag of Words model using CountVectorizer()
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(4,4))
for i in range(len(allData)):
    cv = TfidfVectorizer(ngram_range=(5,5))
    X=allData[i]['human_text']
    Y=allData[i]['class']
    X = cv.fit_transform(X)
    allData[i]=[X,Y]


X = vectorizer.fit_transform(df['human_text'])

In [ ]:
df.shape

In [ ]:
X.shape

In [ ]:
# Creating the weight of Words model using TfidfTransformer()
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_data = TfidfTransformer().fit_transform(X)

In [ ]:
tfidf_data.shape

In [ ]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_data, 
                                                df['class'], 
                                                    test_size = 0.20, 
                                                    random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

Model Creation

**A multinomial naive Bayes classifier will be created. .**

In [ ]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB


In [ ]:
classifier = MultinomialNB(alpha=0.1)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

 **Model performce metrics like the confusion matrix, accuracy, precision, recall and f1 score**

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))


In [ ]:
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report,accuracy_score

In [ ]:
rf = 'Random Forest Classfier'
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_predicted = rf.predict(X_test)
rf_acc_score = accuracy_score(y_test, rf_predicted)   

In [ ]:
print("\n")
print("Accuracy of Random Forest:",rf_acc_score,'\n')
print(classification_report(y_test,rf_predicted))

In [ ]:
dtc= DecisionTreeClassifier()
dtc.fit(X_train,y_train)
y_pred_dtc = dtc.predict(X_test)

In [ ]:
dtc_acc_score = accuracy_score(y_test, y_pred_dtc)

In [ ]:
print("\n")
print("Accuracy of Decision Tree :",dtc_acc_score,'\n')
print(classification_report(y_test,rf_predicted))

In [ ]:
from sklearn import svm

In [ ]:
svm_clf = svm.SVC(kernel='rbf', C=1, random_state=0)
svm_clf.fit(X_train, y_train)
predicted = svm_clf.predict(X_test)
score = svm_clf.score(y_test, y_pred)
svm_score_ = np.mean(score)

print('Accuracy : %.3f' % (svm_score_))

In [ ]:
model_ev = pd.DataFrame({'Model': ['Confusion matrix Classifier','Decision Tree Classifier','Random Forest Classfier'],'Accuracy': [" %.3f" % (accuracy),dtc_acc_score,rf_acc_score]})
model_ev